In [4]:
%pwd

'/home/vinayak.t/IGF-CNN-research-deployment/research'

In [5]:
os.chdir("../")

In [2]:
import os

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int


In [6]:
from src.igfcnnClassifier.constants import *
from src.igfcnnClassifier.utils.common import read_yaml, create_directories

In [7]:

class ConfigurationManager:
    def __init__(self,config_filepath= CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config
    

In [ ]:
import os
import urllib.request as reques

In [13]:
import scipy
from tensorflow.keras.layers import Conv2D,GlobalMaxPooling2D ,MaxPooling2D, Flatten, Dense, Input, Reshape, Lambda

from scipy import fftpack,ifft
def indices(a, func):
    return [i for (i, val) in enumerate(a) if func(val)]

def dftuv(M,N):
    s = []
    d = []
    a = list(range(0, M))
    c = list(range(0, N))
    idx = indices(a, lambda x: x > M / 2)
    bv = indices(a, lambda x: x <= M / 2)
    for i in bv:
        s.append(i)
    for i in idx:
        d.append(i - M)
    u = s + d
    
    b = []
    m = []
    idy = indices(c, lambda x: x > N / 2)
    bw = indices(c, lambda x: x <= N / 2)
    for i in bw:
        b.append(i)
    for i in idy:
        m.append(i - N)
    v = b + m
    [V, U] = np.meshgrid(v, u)
    return [V, U]

def lpfilter(M,N,D0):
    [U, V] = dftuv(M, N)
    D = np.sqrt(U**2 + V**2)
    H = np.exp(-((D**2)/(2*(D0**2))))
    return H



import numpy as np
from scipy import fftpack
import tensorflow as tf



def Fourier_decomposition(img):
    FIBFs = []
    for i in range(1, 6):
        D0 = i * 0.007 * img.shape[1]
        H = lpfilter(img.shape[0], img.shape[1], D0)
        # Convert the input image to a complex tensor with real part as img and imaginary part as zeros
        img_complex = tf.complex(img, tf.zeros_like(img, dtype=tf.float32))

        # Perform the forward Fourier transform using TensorFlow's FFT
        F = tf.signal.fft2d(img_complex)

        # Perform the inverse Fourier transform and take the real part using TensorFlow's IFFT
        LPF_imageSignal_complex = H * F
        LPF_imageSignal = tf.math.real(tf.signal.ifft2d(LPF_imageSignal_complex))

#         lpf = tf.squeeze(LPF_imageSignal, axis=-1)
        
        lpf = LPF_imageSignal[..., 0]
        # Append the LPF_imageSignal to FIBFs without the channel dimension
        FIBFs.append(lpf)

        # Subtract the LPF image signal from the original image
        img = img - LPF_imageSignal

    # Append the final img to FIBFs without the channel dimension

    FIBFs.append(img[..., 0])
    return FIBFs


def feat_concat(img,size,actual_image_size):
    
    # Expand the single-channel image to a three-channel image
    img_expanded = tf.image.resize(img,(actual_image_size,actual_image_size))
    img_pseudo_bgr = tf.concat([img_expanded, img_expanded, img_expanded], axis=-1)
    img_gray = tf.image.rgb_to_grayscale(img_pseudo_bgr)

    print("image_input",img_gray.shape)
    FDM = Fourier_decomposition(img_gray)
    
    FDM[0] = tf.expand_dims(FDM[0], axis=-1)
    FDM[1] = tf.expand_dims(FDM[1], axis=-1)
    FDM[2] = tf.expand_dims(FDM[2], axis=-1)
    FDM[3] = tf.expand_dims(FDM[3], axis=-1)
    FDM[4] = tf.expand_dims(FDM[4], axis=-1)
    FDM[5] = tf.expand_dims(FDM[5], axis=-1)

    FDM0 = tf.image.resize(FDM[0], (size, size), method=tf.image.ResizeMethod.BILINEAR)
    FDM1 = tf.image.resize(FDM[1], (size, size), method=tf.image.ResizeMethod.BICUBIC)
    FDM2 = tf.image.resize(FDM[2], (size, size), method=tf.image.ResizeMethod.BICUBIC)
    FDM3 = tf.image.resize(FDM[3], (size, size), method=tf.image.ResizeMethod.BICUBIC)
    FDM4 = tf.image.resize(FDM[4], (size, size), method=tf.image.ResizeMethod.BICUBIC)
    FDM5 = tf.image.resize(FDM[5], (size, size), method=tf.image.ResizeMethod.BICUBIC)

    FDM_components = tf.stack([FDM1, FDM2, FDM3, FDM4, FDM5], axis=0)

    FDM_components = tf.transpose(FDM_components, perm=[3,1,2,0])
    FDM_components.set_shape([None, size, size, 5])

    print("fdm_shape",FDM_components.shape)
    return FDM_components
    

In [17]:
#TESTING NEW ARCHITCHITEECTURE

import tensorflow as tf
from tensorflow.keras.layers import Conv2D,GlobalMaxPooling2D ,MaxPooling2D, Flatten, Dense, Input, Reshape, Lambda
from tensorflow.keras.models import Model

def IgfCNN(input_shape, num_classes, size):
    inputs = Input(shape=input_shape)    
    img_reshaped = Lambda(lambda x: tf.reshape(x, (-1,) + input_shape[1:]))(inputs)
    processed_images = feat_concat(img_reshaped, size,input_shape[0])

    x = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')(processed_images)
#     x = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')(inputs)

#     x = MaxPooling2D(pool_size=(4, 4), strides=(2, 2))(x)

    x = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(4, 4), strides=(2, 2))(x)
    x = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(4, 4), strides=(1, 1))(x)
    
    x = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(4, 4), strides=(1, 1))(x)
    
    x = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(4, 4), strides=(1, 1))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(16, activation='relu')(x)

    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    summary = model.summary()
    return model





input_shape = (250, 250, 1)
num_classes = 3
size= 50
model = IgfCNN(input_shape, num_classes, size)



image_input (250, 250, 1)
fdm_shape (1, 50, 50, 5)
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 250, 250, 1)]        0         []                            
                                                                                                  
 lambda_4 (Lambda)           (None, 250, 1)               0         ['input_7[0][0]']             
                                                                                                  
 tf.image.resize_22 (TFOpLa  (250, 250, 1)                0         ['lambda_4[0][0]']            
 mbda)                                                                                            
                                                                                                  
 tf.concat_4 (TFOpLambda)    (250, 250, 3

In [12]:

# Step 4: Compile the Model
model.compile(optimizer=SGD(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'SGD' is not defined